In [1]:
import os

repo_dir = os.environ.get("REPO_DIR")
code_dir = os.path.join(repo_dir, "code/")
data_dir = os.path.join(repo_dir, "data/")

os.chdir(code_dir)

import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import pickle
import sklearn 
import sys
import pandas as pd
from importlib import reload

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
import seaborn as sns

from mosaiks.utils.imports import *

# Key prediction functions are here
from analysis.prediction_utils import (X_matrix_to_demeaned_X, df_to_demeaned_y_vars,
                              get_within_perf_from_kfold_dict, cv_solve, solver_kwargs,
                            get_test_idxs)
                              
from analysis.prediction_utils import make_train_pred_scatterplot as make_scatterplot

/home/groups/smhsiang/venv/hdi/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#  Demonstration -- simplified training of HDI model at ADM1 level

This is a greatly simplified version of the `analysis/hdi_and_iwi_model_training.ipynb` notebook.

In this notebook we train an ADM1 model to predict HDI. All data needed to run this demonstration is included in the GitHub repository.

To increase computational efficiency, we do not allow a different ridge hyperparameter on the MOSAIKS and NL features. This is different from what is presented in the primary analysis, although the results are quite similar.

This notebook should run ~5 minutes on a standard laptop computer.

## Read in Xs and Ys -- and subset to training data set

### Read in MOSAIKS/RCF Xs

These daytime image feature files are included in the GitHub repository. They are also available at [mosaiks.org](mosaiks.org).

In [2]:
adm1_X = pd.read_pickle(
(data_dir + "features/mosaiks_features/"
"GDL_ADM1_polygon_X_creation_GHS_POP.p")).drop(columns="GDLCODE")

adm1_X.head()

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_3990,X_3991,X_3992,X_3993,X_3994,X_3995,X_3996,X_3997,X_3998,X_3999
GDLCODE,,,,,,,,,,,,,,,,,,,,,
AFGr101,0.390123,0.678665,0.206719,0.345097,0.548497,0.842432,0.156376,0.381069,0.243269,0.782266,...,0.554838,0.353894,0.964482,0.664171,0.398351,1.397979,2.602631,0.656177,0.792343,0.175051
AFGr102,0.237172,0.580213,0.051948,0.626788,0.372233,0.480073,0.068213,0.089778,0.573527,0.476499,...,0.155166,0.064067,0.270844,0.182315,0.141033,0.754294,0.958891,0.311541,0.530296,0.036294
AFGr103,0.209468,0.437404,0.090435,0.226799,0.313359,0.542390,0.112169,0.217163,0.164701,0.472634,...,0.266113,0.155918,0.542615,0.348209,0.208426,0.796623,1.569982,0.406253,0.473192,0.090176
AFGr104,0.335701,0.688881,0.126453,0.547869,0.501631,0.713768,0.114022,0.221779,0.470572,0.682265,...,0.348477,0.186285,0.607468,0.407335,0.269163,1.134602,1.828728,0.492681,0.702240,0.090141
AFGr105,0.248126,0.545196,0.094966,0.408503,0.386341,0.611326,0.113473,0.197986,0.338978,0.538626,...,0.289186,0.158021,0.537719,0.360405,0.231099,0.970265,1.547651,0.458496,0.607087,0.088386


### Read in Y variables

These `Y` variables (HDI + component indicators) were cleaned and saved in the `/code/analysis/hdi_label_creation.ipynb` notebook. It is not necessary to run this notebook, but we make replication code for data cleaning available.

In [3]:
raw = pd.read_pickle( (data_dir + "int/GDL_HDI/"
                     "HDI_indicators_and_indices_clean.p") ).loc[adm1_X.index]

raw.head()

,ISO_Code,country,year,level,region,continent,Sub-national HDI,Expected years schooling,Mean years schooling,Life expectancy,GNI per capita in thousands of US$ (2011 PPP),pop
GDLCODE,,,,,,,,,,,,
AFGr101,AFG,Afghanistan,2019,Subnat,Central (Kabul Wardak Kapisa Logar Parwan Panj...,Asia/Pacific,0.561,12.898,4.595,65.472,2615.157,
AFGr102,AFG,Afghanistan,2019,Subnat,Central Highlands (Bamyan Daikundi),Asia/Pacific,0.484,12.447,1.715,64.008,1572.594,
AFGr103,AFG,Afghanistan,2019,Subnat,East (Nangarhar Kunar Laghman Nooristan),Asia/Pacific,0.469,9.858,3.071,63.040,1550.047,
AFGr104,AFG,Afghanistan,2019,Subnat,North (Samangan Sar-e-Pul Balkh Jawzjan Faryab),Asia/Pacific,0.507,11.456,2.824,62.396,2491.727,
AFGr105,AFG,Afghanistan,2019,Subnat,North East (Baghlan Takhar Badakhshan Kunduz),Asia/Pacific,0.454,9.288,2.208,62.044,1802.716,


### Read in NL features

These NL features are included in the GitHub repo. They are based off the VIIRS annual average masked data product for the year 2019. Replication code for creating NL features for these administrative polygons is in the `code/analysis/NL_feature_creation_and_other_NL_processing/make_viirs_nl_features_GHS_pop.ipynb.ipynb` notebook.

In [4]:
filepath = (data_dir+f"features/nl_features/GDL_HDI_polygons/"
            "viirs_percentile_binned_feats_GHS_pop_weighted_rasterio_method.p")

nl = pd.read_pickle(filepath).loc[adm1_X.index]

nl.head()

,perc_pixels_in_bin_0,perc_pixels_in_bin_1,perc_pixels_in_bin_2,perc_pixels_in_bin_3,perc_pixels_in_bin_4,perc_pixels_in_bin_5,perc_pixels_in_bin_6,perc_pixels_in_bin_7,perc_pixels_in_bin_8,perc_pixels_in_bin_9,...,perc_pixels_in_bin_11,perc_pixels_in_bin_12,perc_pixels_in_bin_13,perc_pixels_in_bin_14,perc_pixels_in_bin_15,perc_pixels_in_bin_16,perc_pixels_in_bin_17,perc_pixels_in_bin_18,perc_pixels_in_bin_19,perc_pixels_in_bin_20
GDLCODE,,,,,,,,,,,,,,,,,,,,,
AFGr101,0.256695,0.001430,0.004468,0.005563,0.006484,0.007032,0.007237,0.007402,0.008691,0.008267,...,0.009851,0.009057,0.011611,0.012679,0.016088,0.021725,0.036959,0.097207,0.307026,0.157200
AFGr102,0.901871,0.000805,0.011792,0.016059,0.009653,0.004717,0.001293,0.009151,0.008147,0.007157,...,0.005580,0.005521,0.002141,0.002580,0.001939,0.005690,0.001465,0.001509,0.000000,0.000000
AFGr103,0.723645,0.003390,0.010569,0.011927,0.012161,0.012031,0.010685,0.010566,0.010155,0.008597,...,0.009202,0.008768,0.008684,0.009710,0.009419,0.012145,0.021170,0.051236,0.044458,0.001159
AFGr104,0.535552,0.003874,0.017330,0.016035,0.015632,0.012672,0.012772,0.016113,0.013360,0.015471,...,0.021099,0.011580,0.010046,0.014308,0.014031,0.029923,0.039504,0.061986,0.113206,0.009774
AFGr105,0.621222,0.006909,0.023690,0.023718,0.021861,0.022006,0.013871,0.014827,0.014562,0.013012,...,0.011407,0.012378,0.013163,0.014638,0.018651,0.032136,0.032368,0.042387,0.035304,0.000000


### Subset to training data

There is a held out set of test countries. The function `get_test_idxs()` will provide this test set.

In [5]:
train_idxs = raw["ISO_Code"][~raw["ISO_Code"].isin(get_test_idxs())].unique()

In [6]:
# Subset dense_df to training data
train_locs = raw["ISO_Code"].isin(train_idxs)


## Subset Y variables
train_df = raw.loc[train_locs].copy()

# Subset MOSAIKS Xs
train_locs = raw["ISO_Code"].isin(train_idxs)
X_train = adm1_X.loc[train_locs]

# Subset NL Xs
X_nl = nl.loc[train_df.index]

### Create demeaned feature sets

This demeaned Xs are needed for the within-country model.

In [7]:
X_demeaned = X_matrix_to_demeaned_X(X_train)
X_demeaned_nl = X_matrix_to_demeaned_X(X_nl)

### Set some basic solve parameters:

In [8]:
task = 'Sub-national HDI'


In [9]:
def get_idxs_without_adm0_observations(df, country_col = "ISO_Code"):
    """
    For country-demeaned outcomes, we want to remove all the ADM0 level observations. 
    
    In other words, if there is no within-country we shouldn't keep this as training data
    
    """
    col = task
    g = df.groupby("ISO_Code")
    
    country_counts = train_df.groupby(country_col)[col].count() 

    adm0_idxs = country_counts[country_counts == 1].index
    adm1_idxs = df.reset_index().set_index("ISO_Code").drop(adm0_idxs).set_index(df.index.name).index
    
    assert len(adm1_idxs) == len(df)-len(adm0_idxs)
    
    return adm1_idxs

# Begin running solves

###  RCF + NL features, cross country model

In [ ]:
nl_and_rcf_kfold_dict = {}



Y_train = train_df[task]

nl_and_rcf_kfold_dict[task]  = cv_solve(task,
                                        pd.concat([X_train,X_nl],axis=1),# concatenate the rcf and nl data
                                        Y_train,
                                        clip_bounds = [np.min(Y_train), np.max(Y_train)],
                                        country_fold = True)

Sub-national HDI
Training model...

on fold (of 5): 0 1 2 3 

### RCF + NL Demeaned (Demeaned Ys and Demeaned Xs)

In [ ]:
nl_and_rcf_demeaned_kfold_dict = {}


Y_demeaned = df_to_demeaned_y_vars(task, train_df)
    
##Let's drop Y observations that do not have any within country variation. There is nothing to learn from these:
keep_indxs = get_idxs_without_adm0_observations(train_df)
Y_demeaned = Y_demeaned.loc[keep_indxs]
X_demeaned_dropped = X_demeaned.loc[keep_indxs]
X2_demeaned_dropped = X_demeaned_nl.loc[keep_indxs]
    
print("Zero obs dropped =",(len(X_demeaned ) - len(X_demeaned_dropped)))
    
nl_and_rcf_demeaned_kfold_dict[task] = cv_solve(
    task,
    pd.concat([X_demeaned_dropped,X2_demeaned_dropped], axis=1), #concat the RCF and NL Xs
    Y_demeaned,
    clip_bounds = [np.min(Y_demeaned), np.max(Y_demeaned)],
     country_fold = True)